<a href="https://colab.research.google.com/github/elichen/aoc2024/blob/main/Day_17_Chronospatial_Computer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
input = """Register A: 63281501
Register B: 0
Register C: 0

Program: 2,4,1,5,7,5,4,5,0,3,1,6,5,5,3,0"""

In [22]:
def simulate_computer(input_str):
    # Parse initial register values and program
    lines = input_str.strip().split('\n')

    # Initialize registers from input
    r = {
        'A': int(lines[0].split(': ')[1]),
        'B': int(lines[1].split(': ')[1]),
        'C': int(lines[2].split(': ')[1])
    }

    # Parse program into list of integers
    program = [int(x) for x in lines[4].split(': ')[1].split(',')]

    # Instruction pointer
    ip = 0
    out = []

    def combo(operand):
      if operand < 4: return operand
      if operand == 4: return r['A']
      if operand == 5: return r['B']
      if operand == 6: return r['C']
      return None

    while ip < len(program):
        opcode = program[ip]
        operand = program[ip + 1]

        if opcode == 0:   # adv
            num = r['A']
            den = 2**combo(operand)
            r['A'] = num//den
            ip += 2
        elif opcode == 1: # bxl
            r['B'] = r['B'] ^ operand
            ip += 2
        elif opcode == 2: # bst
            r['B'] = combo(operand) % 8
            ip += 2
        elif opcode == 3: # jnz
            if r['A'] == 0:
              ip += 2
            else:
              ip = operand
        elif opcode == 4: # bxc
            r['B'] = r['B'] ^ r['C']
            ip += 2
        elif opcode == 5: # out
            out.append(combo(operand) % 8)
            ip += 2
        elif opcode == 6: # bdv
            num = r['A']
            den = 2**combo(operand)
            r['B'] = num//den
            ip += 2
        elif opcode == 7: # cdv
            num = r['A']
            den = 2**combo(operand)
            r['C'] = num//den
            ip += 2

    return r, out

r,out = simulate_computer(input)
print(",".join(map(str,out)))
r

3,4,3,1,7,6,5,6,0


{'A': 0, 'B': 0, 'C': 0}